In [5]:
import os
import xml.etree.ElementTree as ET
import shutil

In [6]:
def save_license_plate(image_path, output_path, coordinates):
    """
    Save the license plate region based on the bounding box coordinates.
    
    Args:
    - image_path (str): Path to the image containing the license plate.
    - output_path (str): Output path to save the license plate region.
    - coordinates (list): Bounding box coordinates [xmin, ymin, xmax, ymax].
    """
    
    image = cv2.imread(image_path)
    xmin, ymin, xmax, ymax = map(int, coordinates)
    license_plate_region = image[ymin:ymax, xmin:xmax]
    resized_license_plate = cv2.resize(license_plate_region, (192, 96))
    cv2.imwrite(output_path, resized_license_plate)

In [7]:
def convert_annotation(xml_path, img_path, output_label_path):
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # Extract image dimensions
        size = root.find('size')
        w = int(size.find('width').text)
        h = int(size.find('height').text)

        # Open output file
        with open(output_label_path, 'w') as out_file:
            for obj in root.findall('object'):
                # Get class name (assuming 'license_plate' is the class)
                cls_name = obj.find('name').text
                if cls_name == 'license_plate':
                    cls_id = 0  # Assign a class ID (if needed)
                else:
                    continue  # Skip other objects if any

                # Get bounding box coordinates
                bbox = obj.find('bndbox')
                x_min = float(bbox.find('xmin').text)
                y_min = float(bbox.find('ymin').text)
                x_max = float(bbox.find('xmax').text)
                y_max = float(bbox.find('ymax').text)

                # Convert coordinates to YOLO format
                x_center = (x_min + x_max) / (2.0 * w)
                y_center = (y_min + y_max) / (2.0 * h)
                box_width = (x_max - x_min) / w
                box_height = (y_max - y_min) / h

                # Write to output file in YOLO format
                out_file.write(f"{cls_id} {x_center} {y_center} {box_width} {box_height}\n")

            # Save license plate region
                # Generate output path for the license plate image
                # img_filename = os.path.basename(img_path)
                # license_plate_output_path = os.path.join(output_license_plate_dir, img_filename.replace(os.path.splitext(img_filename)[1], f"_license_plate_{cls_id}.jpg"))

            # # Save license plate region
                # save_license_plate(img_path, license_plate_output_path, [x_min, y_min, x_max, y_max])

    except Exception as e:
        print(f'Error processing {xml_path}: {e}')


In [8]:
def main():
    # Directories
    base_dir = './kaggle/working/car-plate-detection'
    img_dir = os.path.join(base_dir, 'images')
    xml_dir = os.path.join(base_dir, 'annotations')
    #output_license_plate_dir = r"C:\Users\fabri\Escritorio\Fifth_try\LicensePlate_text_extraction\kaggle\Bounding_boxes_train"
    
    train_img_output_dir = os.path.join(base_dir, 'train/images')
    train_label_output_dir = os.path.join(base_dir, 'train/labels')
    val_img_output_dir = os.path.join(base_dir, 'val/images')
    val_label_output_dir = os.path.join(base_dir, 'val/labels')

    # Create output directories if they don't exist
    for dir_path in [train_img_output_dir, train_label_output_dir, val_img_output_dir, val_label_output_dir]:
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)

    # Process each XML file
    xml_files = [f for f in os.listdir(xml_dir) if f.endswith('.xml')]
    total_files = len(xml_files)

    train_split = int(0.8 * total_files)  # 80% for training

    for idx, xml_file in enumerate(xml_files): 
        img_file = xml_file.replace('.xml', '')  # File name without extension

        img_extensions = ['.png', '.jpg', '.jpeg']  # Define the image extensions

        # Find the existing image file with one of the specified extensions
        img_found = False
        for ext in img_extensions:
            img_with_ext = img_file + ext
            if os.path.exists(os.path.join(img_dir, img_with_ext)):
                img_file = img_with_ext
                img_found = True
                break

        if img_found:
            if idx < train_split:
                img_output_path = os.path.join(train_img_output_dir, img_file)
                label_output_path = os.path.join(train_label_output_dir, img_file.replace(os.path.splitext(img_file)[1], '.txt'))
            else:
                img_output_path = os.path.join(val_img_output_dir, img_file)
                label_output_path = os.path.join(val_label_output_dir, img_file.replace(os.path.splitext(img_file)[1], '.txt'))

            # Copy image to respective directory
            shutil.copy(os.path.join(img_dir, img_file), img_output_path)

            # Convert annotation
            convert_annotation(os.path.join(xml_dir, xml_file), img_output_path, label_output_path)
        else:
            print(f"No matching image found for {xml_file}.")


main()
